# Modelo de Frecuencia - Contenidos

## Objetivo
Predecir la **frecuencia de siniestros** (número de siniestros) en la cobertura de Contenidos para asegurados que ya siniestrados (clasificación multinomial).

## Pipeline
1. **Fase 1**: Preprocesamiento + Multinomial Logit + Selección Backward + LazyPredict

**Dataset**: Solo registros siniestrados de Contenidos

## 1. Imports y Configuración

In [13]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Preprocesamiento
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# Intentar importar XGBoost y LightGBM (opcionales)
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    HAS_XGB = False
    print("⚠ XGBoost no disponible")

try:
    from lightgbm import LGBMClassifier
    HAS_LGBM = True
except ImportError:
    HAS_LGBM = False
    print("⚠ LightGBM no disponible")

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    make_scorer
)

# LazyPredict
from lazypredict.Supervised import LazyClassifier

# GLM Multinomial Logit
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Carga y Preparación de Datos

In [14]:
# Cargar datos de estudiantes siniestrados
df = pd.read_csv("../data/processed/contenidos_siniestrados.csv")

print(f"Dataset original: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
df.head()

Dataset original: (743, 15)

Columnas: ['año_cursado', 'estudios_area', 'calif_promedio', '2_o_mas_inquilinos', 'distancia_al_campus', 'genero', 'extintor_incendios', 'Gastos_Adicionales_siniestros_num', 'Gastos_Adicionales_siniestros_monto', 'Gastos_Medicos_RC_siniestros_num', 'Gastos_Medicos_RC_siniestros_monto', 'Resp_Civil_siniestros_num', 'Resp_Civil_siniestros_monto', 'Contenidos_siniestros_num', 'Contenidos_siniestros_monto']


,año_cursado,estudios_area,calif_promedio,2_o_mas_inquilinos,distancia_al_campus,genero,extintor_incendios,Gastos_Adicionales_siniestros_num,Gastos_Adicionales_siniestros_monto,Gastos_Medicos_RC_siniestros_num,Gastos_Medicos_RC_siniestros_monto,Resp_Civil_siniestros_num,Resp_Civil_siniestros_monto,Contenidos_siniestros_num,Contenidos_siniestros_monto
0,2do año,Otro,2.45,No,10.08,Femenino,Si,0.00,0.00,0.00,0.00,0.00,0.00,1.00,990.60
1,4to año,Ciencias,4.69,Si,0.00,Masculino,Si,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1172.94
2,2do año,Administracion,6.46,Si,0.00,Masculino,Si,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2106.47
3,3er año,Ciencias,8.02,Si,9.15,Femenino,Si,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1433.56
4,4to año,Ciencias,8.10,Si,0.00,Femenino,No,0.00,0.00,0.00,0.00,0.00,0.00,1.00,500.61


In [15]:
# Eliminar columna de monto (no se usa en modelo de frecuencia)
df = df.drop('Contenidos_siniestros_monto', axis=1)

# Eliminar otras coberturas (no son features relevantes para este modelo)
df = df.drop([
    'Gastos_Medicos_RC_siniestros_num', 'Gastos_Medicos_RC_siniestros_monto',
    'Resp_Civil_siniestros_num', 'Resp_Civil_siniestros_monto',
    'Gastos_Adicionales_siniestros_num', 'Gastos_Adicionales_siniestros_monto'
], axis=1)

print(f"\nDataset preparado: {df.shape}")
print(f"\nDistribución variable objetivo (frecuencia):")
print(df['Contenidos_siniestros_num'].value_counts().sort_index())
print(f"\nEstadísticas de frecuencia:")
print(df['Contenidos_siniestros_num'].describe())


Dataset preparado: (743, 8)

Distribución variable objetivo (frecuencia):
Contenidos_siniestros_num
1.00    702
2.00     38
3.00      3
Name: count, dtype: int64

Estadísticas de frecuencia:
count   743.00
mean      1.06
std       0.25
min       1.00
25%       1.00
50%       1.00
75%       1.00
max       3.00
Name: Contenidos_siniestros_num, dtype: float64


## 3. Análisis Exploratorio Rápido

In [16]:
# Identificar tipos de variables
numeric_features = ['calif_promedio', 'distancia_al_campus']
categorical_features = ['año_cursado', 'estudios_area', '2_o_mas_inquilinos', 'genero', 'extintor_incendios']

print("Variables numéricas:", numeric_features)
print("Variables categóricas:", categorical_features)
print(f"\nTotal features: {len(numeric_features) + len(categorical_features)}")

# Verificar valores faltantes
print(f"\nValores faltantes:\n{df.isnull().sum()}")

Variables numéricas: ['calif_promedio', 'distancia_al_campus']
Variables categóricas: ['año_cursado', 'estudios_area', '2_o_mas_inquilinos', 'genero', 'extintor_incendios']

Total features: 7

Valores faltantes:
año_cursado                  0
estudios_area                0
calif_promedio               0
2_o_mas_inquilinos           0
distancia_al_campus          0
genero                       0
extintor_incendios           0
Contenidos_siniestros_num    0
dtype: int64


In [17]:
# Estadísticas descriptivas - Frecuencia promedio por categoría
print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS - FRECUENCIA PROMEDIO POR CATEGORÍA")
print("=" * 80)

for var in categorical_features:
    print(f"\n{var}:")
    tabla = df.groupby(var)['Contenidos_siniestros_num'].agg(['mean', 'count', 'sum'])
    print(tabla.round(3))

ESTADÍSTICAS DESCRIPTIVAS - FRECUENCIA PROMEDIO POR CATEGORÍA

año_cursado:
             mean  count    sum
año_cursado                    
1er año      1.09    185 201.00
2do año      1.04    183 191.00
3er año      1.07    171 183.00
4to año      1.03    144 149.00
posgrado     1.05     60  63.00

estudios_area:
                mean  count    sum
estudios_area                     
Administracion  1.06    208 221.00
Ciencias        1.03    188 193.00
Humanidades     1.05    171 179.00
Otro            1.10    176 194.00

2_o_mas_inquilinos:
                    mean  count    sum
2_o_mas_inquilinos                    
No                  1.04    490 512.00
Si                  1.09    253 275.00

genero:
              mean  count    sum
genero                          
Femenino      1.09    323 351.00
Masculino     1.04    347 360.00
No respuesta  1.03     29  30.00
Otro          1.04     44  46.00

extintor_incendios:
                    mean  count    sum
extintor_incendios            

## 4. Preprocesamiento y Split de Datos

In [18]:
# Separar features y target
X = df.drop('Contenidos_siniestros_num', axis=1)
y = df['Contenidos_siniestros_num']

# Split train/test estratificado 80/20 (estratificado por frecuencia)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print(f"Train set: {X_train.shape} | Frecuencia promedio: {y_train.mean():.4f}")
print(f"Test set:  {X_test.shape} | Frecuencia promedio: {y_test.mean():.4f}")
print(f"\nDistribución train:\n{y_train.value_counts().sort_index()}")
print(f"\nDistribución test:\n{y_test.value_counts().sort_index()}")

Train set: (594, 7) | Frecuencia promedio: 1.0606
Test set:  (149, 7) | Frecuencia promedio: 1.0537

Distribución train:
Contenidos_siniestros_num
1.00    561
2.00     30
3.00      3
Name: count, dtype: int64

Distribución test:
Contenidos_siniestros_num
1.00    141
2.00      8
Name: count, dtype: int64


In [19]:
# Pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='drop'
)

# Ajustar y transformar datos
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Obtener nombres de features después de one-hot encoding
feature_names_num = numeric_features
feature_names_cat = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_names = list(feature_names_num) + list(feature_names_cat)

# Convertir a DataFrame (mantener índices originales para alinear con y_train/y_test)
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

print(f"\nFeatures después de preprocesamiento: {len(feature_names)}")
print(f"Shape X_train: {X_train_df.shape}")
print(f"Shape X_test: {X_test_df.shape}")


Features después de preprocesamiento: 14
Shape X_train: (594, 14)
Shape X_test: (149, 14)


## 5. Fase 1 - Multinomial Logit con Selección Backward de Variables

In [20]:
# Modelo Multinomial Logit completo para referencia
X_train_const = sm.add_constant(X_train_df)

# Usar MNLogit para clasificación multinomial de frecuencia
from statsmodels.discrete.discrete_model import MNLogit

mnlogit_full = MNLogit(y_train, X_train_const)
result_full = mnlogit_full.fit(disp=0)

print("=" * 80)
print("MODELO MULTINOMIAL LOGIT COMPLETO (todas las variables)")
print("=" * 80)
print(result_full.summary())

MODELO MULTINOMIAL LOGIT COMPLETO (todas las variables)
                              MNLogit Regression Results                             
Dep. Variable:     Contenidos_siniestros_num   No. Observations:                  594
Model:                               MNLogit   Df Residuals:                      564
Method:                                  MLE   Df Model:                           28
Date:                       Fri, 10 Oct 2025   Pseudo R-squ.:                     nan
Time:                               00:08:43   Log-Likelihood:                    nan
converged:                              True   LL-Null:                       -137.50
Covariance Type:                   nonrobust   LLR p-value:                       nan
Contenidos_siniestros_num=2       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                              nan        nan        nan    

In [21]:
# Selección Backward basada en p-valores con Multinomial Logit
from statsmodels.discrete.discrete_model import MNLogit

def backward_elimination(X, y, significance_level=0.05):
    """
    Realiza selección backward de variables basada en p-valores del Multinomial Logit.
    """
    features = list(X.columns)
    
    while len(features) > 0:
        X_const = sm.add_constant(X[features])
        model = MNLogit(y, X_const).fit(disp=0)
        
        # Obtener p-valores excluyendo la constante
        # En MNLogit, los p-valores están organizados por clase
        p_values = model.pvalues.iloc[:, 1:]  # Excluir constante
        
        # Obtener el máximo p-valor entre todas las clases
        if p_values.empty or len(p_values.columns) == 0:
            print("⚠ No hay más features para evaluar")
            break
        
        max_p_values = p_values.max(axis=0)  # Máximo p-valor por feature
        max_p_value = max_p_values.max()
        
        # Si el p-valor máximo es menor o igual al nivel de significancia, todas son significativas
        if pd.isna(max_p_value) or max_p_value <= significance_level:
            break
            
        feature_to_remove = max_p_values.idxmax()
        features.remove(feature_to_remove)
        print(f"Eliminando: {feature_to_remove} (p-valor: {max_p_value:.4f})")
    
    return features, model

print("=" * 80)
print("SELECCIÓN BACKWARD DE VARIABLES (α = 0.05)")
print("=" * 80)
print()

selected_features, mnlogit_backward = backward_elimination(X_train_df, y_train, significance_level=0.05)

print()
print("=" * 80)
print(f"VARIABLES SELECCIONADAS: {len(selected_features)}")
print("=" * 80)
for feat in selected_features:
    print(f"  • {feat}")
print()
print(mnlogit_backward.summary())

SELECCIÓN BACKWARD DE VARIABLES (α = 0.05)


VARIABLES SELECCIONADAS: 14
  • calif_promedio
  • distancia_al_campus
  • año_cursado_2do año
  • año_cursado_3er año
  • año_cursado_4to año
  • año_cursado_posgrado
  • estudios_area_Ciencias
  • estudios_area_Humanidades
  • estudios_area_Otro
  • 2_o_mas_inquilinos_Si
  • genero_Masculino
  • genero_No respuesta
  • genero_Otro
  • extintor_incendios_Si

                              MNLogit Regression Results                             
Dep. Variable:     Contenidos_siniestros_num   No. Observations:                  594
Model:                               MNLogit   Df Residuals:                      564
Method:                                  MLE   Df Model:                           28
Date:                       Fri, 10 Oct 2025   Pseudo R-squ.:                     nan
Time:                               00:08:43   Log-Likelihood:                    nan
converged:                              True   LL-Null:                      

In [22]:
# Expandir variables categóricas completas
def expand_categorical_features(selected_features, all_feature_names, categorical_features):
    selected_cat_vars = set()
    for feat in selected_features:
        for cat_var in categorical_features:
            if feat.startswith(cat_var + '_'):
                selected_cat_vars.add(cat_var)
                break
    
    final_features = []
    for feat in selected_features:
        is_categorical = any(feat.startswith(cat_var + '_') for cat_var in categorical_features)
        if not is_categorical:
            final_features.append(feat)
    
    for feat in all_feature_names:
        for cat_var in selected_cat_vars:
            if feat.startswith(cat_var + '_') and feat not in final_features:
                final_features.append(feat)
                break
    
    return sorted(final_features)

final_selected_features = expand_categorical_features(selected_features, feature_names, categorical_features)

print("=" * 80)
print("FEATURES FINALES (con todos los niveles de categóricas)")
print("=" * 80)
print(f"\nTotal features: {len(final_selected_features)}")
print("\nFeatures seleccionadas:")
for feat in final_selected_features:
    indicator = "✓" if feat in selected_features else "+"
    print(f"  {indicator} {feat}")

X_train_selected = X_train_df[final_selected_features]
X_test_selected = X_test_df[final_selected_features]

print(f"\nShape final - Train: {X_train_selected.shape} | Test: {X_test_selected.shape}")

FEATURES FINALES (con todos los niveles de categóricas)

Total features: 14

Features seleccionadas:
  ✓ 2_o_mas_inquilinos_Si
  ✓ año_cursado_2do año
  ✓ año_cursado_3er año
  ✓ año_cursado_4to año
  ✓ año_cursado_posgrado
  ✓ calif_promedio
  ✓ distancia_al_campus
  ✓ estudios_area_Ciencias
  ✓ estudios_area_Humanidades
  ✓ estudios_area_Otro
  ✓ extintor_incendios_Si
  ✓ genero_Masculino
  ✓ genero_No respuesta
  ✓ genero_Otro

Shape final - Train: (594, 14) | Test: (149, 14)


## 6. Fase 1 - LazyPredict para Identificar Mejores Algoritmos

In [23]:
# Ejecutar LazyPredict para clasificación multinomial
print("=" * 80)
print("EJECUTANDO LAZYPREDICT (CLASIFICACIÓN MULTINOMIAL)")
print("=" * 80)
print(f"\nDataset: {X_train_selected.shape[0]} muestras, {X_train_selected.shape[1]} features")
print("Esto puede tomar varios minutos...\n")

clf = LazyClassifier(verbose=0, ignore_warnings=True, predictions=True, random_state=42)
models, predictions = clf.fit(X_train_selected, X_test_selected, y_train, y_test)

print("\n" + "=" * 80)
print("RESULTADOS DE LAZYPREDICT")
print("=" * 80)
print()
print(models)

EJECUTANDO LAZYPREDICT (CLASIFICACIÓN MULTINOMIAL)

Dataset: 594 muestras, 14 features
Esto puede tomar varios minutos...



  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000147 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 319
[LightGBM] [Info] Number of data points in the train set: 594, number of used features: 14
[LightGBM] [Info] Start training from score -0.057158
[LightGBM] [Info] Start training from score -2.985682
[LightGBM] [Info] Start training from score -5.288267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

In [24]:
# Top 5 modelos por F1 Score (mayor es mejor para clasificación)
print("=" * 80)
print("TOP 5 MODELOS POR F1 SCORE")
print("=" * 80)
print()

top_5_models = models.nlargest(5, 'F1 Score')
print(top_5_models)

# Agregar modelos adicionales solicitados explícitamente
modelos_adicionales = ['LogisticRegression', 'RandomForestClassifier', 'XGBClassifier', 'GradientBoostingClassifier']
modelos_para_optimizar = list(set(list(top_5_models.index) + modelos_adicionales))

print("\n" + "=" * 80)
print("MODELOS SELECCIONADOS PARA FASE 2 (GridSearchCV)")
print("=" * 80)
print(f"\nTotal modelos a optimizar: {len(modelos_para_optimizar)}")
print("\nDe LazyPredict (Top 5):")
for i, model_name in enumerate(top_5_models.index, 1):
    f1 = top_5_models.loc[model_name, 'F1 Score']
    acc = top_5_models.loc[model_name, 'Accuracy']
    print(f"{i}. {model_name} - F1: {f1:.4f} | Accuracy: {acc:.4f}")

print("\nModelos adicionales solicitados:")
for model_name in modelos_adicionales:
    if model_name in models.index:
        f1 = models.loc[model_name, 'F1 Score']
        acc = models.loc[model_name, 'Accuracy']
        print(f"  • {model_name} - F1: {f1:.4f} | Accuracy: {acc:.4f}")
    else:
        print(f"  • {model_name} - (no evaluado en LazyPredict)")

TOP 5 MODELOS POR F1 SCORE

                        Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                    
AdaBoostClassifier          0.95               0.50     0.50      0.92   
CalibratedClassifierCV      0.95               0.50     0.50      0.92   
DummyClassifier             0.95               0.50     0.50      0.92   
BernoulliNB                 0.95               0.50     0.50      0.92   
BaggingClassifier           0.95               0.50     0.50      0.92   

                        Time Taken  
Model                               
AdaBoostClassifier            0.09  
CalibratedClassifierCV        0.05  
DummyClassifier               0.01  
BernoulliNB                   0.01  
BaggingClassifier             0.03  

MODELOS SELECCIONADOS PARA FASE 2 (GridSearchCV)

Total modelos a optimizar: 9

De LazyPredict (Top 5):
1. AdaBoostClassifier - F1: 0.9202 | Accuracy: 0.9463
2. CalibratedClassifierCV - F1: 0.9

---
## Fin de Fase 1

**Fase 1 completada exitosamente:**
1. ✅ Preprocesamiento con StandardScaler + OneHotEncoder
2. ✅ Multinomial Logit con todas las variables
3. ✅ Selección Backward de variables (α = 0.05)
4. ✅ Expansión de variables categóricas completas
5. ✅ LazyPredict para identificar mejores algoritmos

**Próximos pasos (Fase 2):**
- Optimización de hiperparámetros con GridSearchCV
- Evaluación en conjunto de test
- Selección del mejor modelo

---
## FASE 2: Optimización de Hiperparámetros con GridSearchCV

**Objetivo:** Optimizar los mejores modelos identificados en Fase 1 usando validación cruzada estratificada
**Métrica de optimización:** F1-Score (weighted) - mayor es mejor
**Métricas de evaluación:** F1-Score, Accuracy, Precision, Recall, Gini

In [25]:
# Función de evaluación Gini para clasificación multinomial
def gini_coefficient_multiclass(y_true, y_pred_proba):
    """
    Calcula el coeficiente de Gini promedio para clasificación multinomial.
    Usa One-vs-Rest para cada clase.
    """
    from sklearn.preprocessing import label_binarize
    
    classes = np.unique(y_true)
    y_true_bin = label_binarize(y_true, classes=classes)
    
    gini_scores = []
    for i in range(len(classes)):
        try:
            auc = roc_auc_score(y_true_bin[:, i], y_pred_proba[:, i])
            gini = 2 * auc - 1
            gini_scores.append(gini)
        except:
            continue
    
    return np.mean(gini_scores) if gini_scores else np.nan

print("✓ Funciones de evaluación definidas")

✓ Funciones de evaluación definidas


In [26]:
# Grillas de hiperparámetros con parámetros de suavizado/regularización
param_grids = {
    'LogisticRegression': {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  # Inverso de regularización
        'penalty': ['l1', 'l2'],
        'solver': ['saga'],  # Soporta l1 y l2 para multinomial
        'multi_class': ['multinomial'],
        'max_iter': [1000],
        'class_weight': ['balanced', None],
        'random_state': [42]
    },
    
    'DecisionTreeClassifier': {
        'max_depth': [3, 5, 7],
        'min_samples_split': [10, 20, 50],
        'min_samples_leaf': [5, 10, 20],
        'max_features': ['sqrt', 'log2'],
        'criterion': ['gini', 'entropy'],
        'class_weight': ['balanced', None],
        'ccp_alpha': [0.0, 0.01, 0.05],  # Poda de complejidad
        'random_state': [42]
    },
    
    'RandomForestClassifier': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7, None],
        'min_samples_split': [10, 20, 50],
        'min_samples_leaf': [5, 10, 20],
        'max_features': ['sqrt', 'log2'],
        'class_weight': ['balanced', 'balanced_subsample', None],
        'ccp_alpha': [0.0, 0.01],  # Poda
        'random_state': [42]
    },
    
    'GradientBoostingClassifier': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'min_samples_split': [10, 20],
        'min_samples_leaf': [5, 10],
        'subsample': [0.7, 0.8, 1.0],
        'max_features': ['sqrt', 'log2'],
        'ccp_alpha': [0.0, 0.01],  # Poda
        'random_state': [42]
    },
    
    'AdaBoostClassifier': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.5, 1.0],
        'algorithm': ['SAMME'],
        'estimator': [
            DecisionTreeClassifier(max_depth=1, random_state=42),
            DecisionTreeClassifier(max_depth=2, random_state=42),
            DecisionTreeClassifier(max_depth=3, random_state=42)
        ],
        'random_state': [42]
    },
    
    'XGBClassifier': {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [50, 100, 200],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0],
        'reg_alpha': [0, 0.1, 1.0],  # L1 regularization
        'reg_lambda': [1, 2, 5],  # L2 regularization
        'random_state': [42]
    },
    
    'KNeighborsClassifier': {
        'n_neighbors': [3, 5, 7, 10],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    
    'SVC': {
        'C': [0.1, 1.0, 10.0],  # Regularización
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale', 'auto'],
        'class_weight': ['balanced', None],
        'probability': [True],  # Para predict_proba
        'random_state': [42]
    },
    
    'LinearDiscriminantAnalysis': {
        'solver': ['svd', 'lsqr', 'eigen'],
        'shrinkage': [None, 0.1, 0.5, 0.9, 'auto']
    },
    
    'QuadraticDiscriminantAnalysis': {
        'reg_param': [0.0, 0.1, 0.3, 0.5, 0.7]  # Regularización
    }
}

model_mapping = {
    'LogisticRegression': LogisticRegression(random_state=42),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
    'RandomForestClassifier': RandomForestClassifier(random_state=42),
    'GradientBoostingClassifier': GradientBoostingClassifier(random_state=42),
    'AdaBoostClassifier': AdaBoostClassifier(random_state=42),
    'XGBClassifier': XGBClassifier(random_state=42, eval_metric='mlogloss'),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'SVC': SVC(random_state=42),
    'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(),
    'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis()
}

print("✓ Grillas con parámetros de regularización/suavizado:")
print("  - LogisticRegression: C (regularización L1/L2) + class_weight")
print("  - DecisionTreeClassifier: poda (ccp_alpha) + class_weight")
print("  - RandomForestClassifier: poda + min_samples + class_weight")
print("  - GradientBoostingClassifier: learning_rate + subsample + poda")
print("  - AdaBoostClassifier: learning_rate + estimator")
print("  - XGBClassifier: reg_alpha/lambda + learning_rate")
print("  - KNeighborsClassifier: n_neighbors + weights")
print("  - SVC: C (regularización) + class_weight")
print("  - LinearDiscriminantAnalysis: shrinkage")
print("  - QuadraticDiscriminantAnalysis: reg_param")

✓ Grillas con parámetros de regularización/suavizado:
  - LogisticRegression: C (regularización L1/L2) + class_weight
  - DecisionTreeClassifier: poda (ccp_alpha) + class_weight
  - RandomForestClassifier: poda + min_samples + class_weight
  - GradientBoostingClassifier: learning_rate + subsample + poda
  - AdaBoostClassifier: learning_rate + estimator
  - XGBClassifier: reg_alpha/lambda + learning_rate
  - KNeighborsClassifier: n_neighbors + weights
  - SVC: C (regularización) + class_weight
  - LinearDiscriminantAnalysis: shrinkage
  - QuadraticDiscriminantAnalysis: reg_param


In [27]:
# GridSearchCV con validación cruzada
print("=" * 80)
print("GRID SEARCH CON VALIDACIÓN CRUZADA")
print("=" * 80)
print()

# Scorer F1-weighted (apropiado para desbalanceo de clases)
f1_scorer = make_scorer(f1_score, average='weighted')
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

optimized_models = {}
grid_results = {}

for i, model_name in enumerate(modelos_para_optimizar, 1):
    print(f"\n{'='*80}")
    print(f"[{i}/{len(modelos_para_optimizar)}] Optimizando: {model_name}")
    print(f"{'='*80}")
    
    if model_name not in param_grids or model_name not in model_mapping:
        print(f"⚠ No hay grilla para {model_name}, saltando...")
        continue
    
    base_model = model_mapping[model_name]
    param_grid = param_grids[model_name]
    
    n_combinations = np.prod([len(v) if isinstance(v, list) else 1 for v in param_grid.values()])
    print(f"Combinaciones: {int(n_combinations)} | Total fits: {int(5 * n_combinations)}")
    
    try:
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grid,
            cv=cv,
            scoring=f1_scorer,  # Optimiza F1-score weighted
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train_selected, y_train)
        
        optimized_models[model_name] = grid_search.best_estimator_
        grid_results[model_name] = {
            'best_params': grid_search.best_params_,
            'best_score_cv': grid_search.best_score_
        }
        
        print(f"✓ F1-Score CV: {grid_search.best_score_:.4f}")
        print(f"Mejores parámetros: {grid_search.best_params_}")
        
    except Exception as e:
        print(f"✗ Error: {str(e)}")

print(f"\n{'='*80}")
print(f"Completado: {len(optimized_models)}/{len(modelos_para_optimizar)} modelos")

GRID SEARCH CON VALIDACIÓN CRUZADA


[1/9] Optimizando: LogisticRegression
Combinaciones: 24 | Total fits: 120
✓ F1-Score CV: 0.9175
Mejores parámetros: {'C': 0.001, 'class_weight': None, 'max_iter': 1000, 'multi_class': 'multinomial', 'penalty': 'l1', 'random_state': 42, 'solver': 'saga'}

[2/9] Optimizando: BernoulliNB
⚠ No hay grilla para BernoulliNB, saltando...

[3/9] Optimizando: GradientBoostingClassifier
Combinaciones: 1296 | Total fits: 6480
✓ F1-Score CV: 0.9255
Mejores parámetros: {'ccp_alpha': 0.0, 'learning_rate': 0.1, 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 100, 'random_state': 42, 'subsample': 0.8}

[4/9] Optimizando: DummyClassifier
⚠ No hay grilla para DummyClassifier, saltando...

[5/9] Optimizando: AdaBoostClassifier
Combinaciones: 36 | Total fits: 180
✓ F1-Score CV: 0.9195
Mejores parámetros: {'algorithm': 'SAMME', 'estimator': DecisionTreeClassifier(max_depth=3, random_state=42), 'learning_rate': 0.1, 

In [28]:
# Evaluación en test set
print("=" * 80)
print("EVALUACIÓN EN TEST SET")
print("=" * 80)

test_results = []

for model_name, model in optimized_models.items():
    print(f"\n{'='*80}")
    print(f"Evaluando: {model_name}")
    print(f"{'='*80}")
    
    # Predicciones en test
    y_pred = model.predict(X_test_selected)
    
    # Predicciones de probabilidad (si el modelo lo soporta)
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_selected)
    elif hasattr(model, 'decision_function'):
        y_pred_proba = None  # No todas las decisiones son probabilidades
    else:
        y_pred_proba = None
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # Calcular Gini si hay probabilidades
    try:
        if y_pred_proba is not None:
            gini = gini_coefficient_multiclass(y_test, y_pred_proba)
        else:
            gini = np.nan
    except:
        gini = np.nan
    
    test_results.append({
        'Modelo': model_name,
        'F1_CV': grid_results[model_name]['best_score_cv'],
        'F1_Test': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Gini': gini
    })
    
    print(f"F1 (CV): {grid_results[model_name]['best_score_cv']:.4f} | F1 (Test): {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
    print(f"Gini: {gini:.4f}")
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nMatriz de Confusión:")
    print(cm)
    
    # Distribución de predicciones
    print(f"\nDistribución de predicciones:")
    pred_dist = pd.Series(y_pred).value_counts().sort_index()
    real_dist = pd.Series(y_test).value_counts().sort_index()
    print(f"  Predicho: {dict(pred_dist)}")
    print(f"  Real:     {dict(real_dist)}")

df_results = pd.DataFrame(test_results).sort_values('F1_Test', ascending=False)

print(f"\n{'='*80}")
print("RESUMEN DE RESULTADOS (ordenado por F1-Score Test)")
print(f"{'='*80}")
print()
print(df_results.to_string(index=False))

EVALUACIÓN EN TEST SET

Evaluando: LogisticRegression
F1 (CV): 0.9175 | F1 (Test): 0.9202
Accuracy: 0.9463 | Precision: 0.8955 | Recall: 0.9463
Gini: 0.0000

Matriz de Confusión:
[[141   0]
 [  8   0]]

Distribución de predicciones:
  Predicho: {1.0: np.int64(149)}
  Real:     {1.0: np.int64(141), 2.0: np.int64(8)}

Evaluando: GradientBoostingClassifier
F1 (CV): 0.9255 | F1 (Test): 0.9353
Accuracy: 0.9530 | Precision: 0.9552 | Recall: 0.9530
Gini: -0.1348

Matriz de Confusión:
[[141   0]
 [  7   1]]

Distribución de predicciones:
  Predicho: {1.0: np.int64(148), 2.0: np.int64(1)}
  Real:     {1.0: np.int64(141), 2.0: np.int64(8)}

Evaluando: AdaBoostClassifier
F1 (CV): 0.9195 | F1 (Test): 0.9353
Accuracy: 0.9530 | Precision: 0.9552 | Recall: 0.9530
Gini: -0.1658

Matriz de Confusión:
[[141   0]
 [  7   1]]

Distribución de predicciones:
  Predicho: {1.0: np.int64(148), 2.0: np.int64(1)}
  Real:     {1.0: np.int64(141), 2.0: np.int64(8)}

Evaluando: RandomForestClassifier
F1 (CV): 0.917

In [36]:
  print("=" * 80)
  print("REENTRENAMIENTO CON DATASET COMPLETO")
  print("=" * 80)
  print()

  # Combinar train y test
  X_full = pd.concat([X_train_selected, X_test_selected])
  y_full = pd.concat([y_train, y_test])

  print(f"Dataset completo: {X_full.shape}")
  print(f"Distribución de frecuencia:\n{y_full.value_counts().sort_index()}")

  # Obtener los mejores parámetros del
  GradientBoostingClassifier
  best_params_gbc = grid_results['GradientBoostingClassifier']['best_params']
  print(f"\nMejores parámetros del GradientBoostingClassifier:")
  for param, value in best_params_gbc.items():
      print(f"  {param}: {value}")

  # Crear y entrenar modelo con todos los datos
  gbc_final = GradientBoostingClassifier(**best_params_gbc)
  gbc_final.fit(X_full, y_full)

  print(f"\n✓ Modelo reentrenado con {len(X_full)} registros")

  # Celda 2: Exportación del modelo
  import pickle
  import os

  # Crear directorio models si no existe
  os.makedirs('../models', exist_ok=True)

  # Preparar objeto para exportar
  modelo_export = {
      'modelo': gbc_final,
      'preprocessor': preprocessor,
      'features_seleccionadas': final_selected_features,
      'metricas': {
          'F1_CV': grid_results['GradientBoostingClassifier']['best_score_cv'],
          'F1_Test': df_results[df_results['Modelo'] == 'GradientBoostingClassifier']['F1_Test'].values[0],
          'Accuracy': df_results[df_results['Modelo'] == 'GradientBoostingClassifier']['Accuracy'].values[0],
          'Precision': df_results[df_results['Modelo'] == 'GradientBoostingClassifier']['Precision'].values[0],
          'Recall': df_results[df_results['Modelo'] == 'GradientBoostingClassifier']['Recall'].values[0],
          'Gini': df_results[df_results['Modelo'] == 'GradientBoostingClassifier']['Gini'].values[0]
      },
      'mejores_parametros': best_params_gbc
  }

  # Exportar modelo
  model_path = '../models/frecuencia_contenidos.pkl'
  with open(model_path, 'wb') as f:
      pickle.dump(modelo_export, f)

  print("=" * 80)
  print("MODELO EXPORTADO EXITOSAMENTE")
  print("=" * 80)
  print(f"\nRuta: {model_path}")
  print(f"\nModelo: GradientBoostingClassifier")
  print(f"Mejores parámetros: {modelo_export['mejores_parametros']}")
  print(f"\nMétricas en Test Set:")
  print(f"  F1-Score: {modelo_export['metricas']['F1_Test']:.4f}")
  print(f"  Accuracy: {modelo_export['metricas']['Accuracy']:.4f}")
  print(f"  Precision: {modelo_export['metricas']['Precision']:.4f}")
  print(f"  Recall: {modelo_export['metricas']['Recall']:.4f}")
  print(f"  Gini: {modelo_export['metricas']['Gini']:.4f}")
  print(f"\nContenido del archivo:")
  print(f"  • modelo: GradientBoostingClassifier optimizado")
  print(f"  • preprocessor: ColumnTransformer (StandardScaler + OneHotEncoder)")
  print(f"  • features_seleccionadas: {final_selected_features}")
  print(f"  • metricas: diccionario con todas las métricas")
  print(f"  • mejores_parametros: {modelo_export['mejores_parametros']}")

REENTRENAMIENTO CON DATASET COMPLETO

Dataset completo: (743, 14)
Distribución de frecuencia:
Contenidos_siniestros_num
1.00    702
2.00     38
3.00      3
Name: count, dtype: int64

Mejores parámetros del GradientBoostingClassifier:
  ccp_alpha: 0.0
  learning_rate: 0.1
  max_depth: 7
  max_features: sqrt
  min_samples_leaf: 5
  min_samples_split: 10
  n_estimators: 100
  random_state: 42
  subsample: 0.8

✓ Modelo reentrenado con 743 registros
MODELO EXPORTADO EXITOSAMENTE

Ruta: ../models/frecuencia_contenidos.pkl

Modelo: GradientBoostingClassifier
Mejores parámetros: {'ccp_alpha': 0.0, 'learning_rate': 0.1, 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 10, 'n_estimators': 100, 'random_state': 42, 'subsample': 0.8}

Métricas en Test Set:
  F1-Score: 0.9353
  Accuracy: 0.9530
  Precision: 0.9552
  Recall: 0.9530
  Gini: -0.1348

Contenido del archivo:
  • modelo: GradientBoostingClassifier optimizado
  • preprocessor: ColumnTransformer (Standard